In [76]:
import pandas as pd
from datasets import Dataset
from keras.src.utils import split_dataset
from pandas.io.sas.sas_constants import column_name_text_subheader_length
from pyarrow.dataset import dataset
from torchgen.executorch.api.et_cpp import return_type

df= messages= pd.read_csv(r'C:\Users\Suman Kar\Downloads\SMSSpamCollection (1).txt',sep='\t',names=['label','message'])

df['label']=df['label'].map({'ham':0,'spam':1})
dataset= Dataset.from_pandas(df)

print(dataset)

Dataset({
    features: ['label', 'message'],
    num_rows: 5572
})


TFDistilBertForSequenceClassification{FOR SENTIMENT ANALYSIS}


In [78]:
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer

# Load DistilBERT model for binary classification
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [81]:
import tensorflow as tf
from transformers import AutoTokenizer

tokenizers= AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["message"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/5572 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'message', 'input_ids', 'attention_mask'],
    num_rows: 5572
})


In [85]:
split_dataset = dataset.train_test_split(test_size=0.2,seed=42)

train_dataset=split_dataset["train"]
eval_dataset=split_dataset["test"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['label', 'message'],
    num_rows: 4457
})
Dataset({
    features: ['label', 'message'],
    num_rows: 1115
})


In [88]:

import tensorflow as tf

def convert_to_tf_dataset(dataset, batch_size=8):

    dataset.set_format(type="tensorflow",columns=["input_ids", "attention_mask", "label"])

    return dataset.to_tf_dataset(
           columns=["input_ids", "attention_mask"],
           label_cols=["label"],
           shuffle=True,
           batch_size=batch_size
    )


train_dataset = convert_to_tf_dataset(tokenized_datasets["train"])
eval_dataset = convert_to_tf_dataset(tokenized_datasets["test"])


KeyError: "Column train not in the dataset. Current columns in the dataset: ['label', 'message', 'input_ids', 'attention_mask']"